In [9]:
from __future__ import annotations
from typing import Tuple, List, Dict, KeysView, Iterable
from random import uniform
from gurobipy import Model, tupledict, GurobiError, GRB
from graph_tool import Graph, EdgePropertyMap
from graph_tool.generation import complete_graph
from graph_tool.flow import boykov_kolmogorov_max_flow, min_st_cut

In [10]:
Vertex = int
Arc = Tuple[Vertex, Vertex]
Tour = List[Vertex]

class TSPIntance:
    n: int
    x: List[float]
    y: List[float]
    cost: Dict[Arc, float]

    def __init__(self, x: List[float], y: List[float]):
        assert len(x) == len(y), "X and Y coordinate lists must have the same length"

        self.n = len(x)
        self.x = x
        self.y = y
        self.cost = {
            (i, j): ((self.x[i] - self.x[j])**2 + (self.y[i] - self.y[j])**2)**0.5
            for i in self.vertices()
            for j in self.vertices()
            if i != j
        }

    def vertices(self) -> Iterable[Vertex]:
        return range(self.n)
    
    def arcs(self) -> KeysView:
        return self.cost.keys()

    @staticmethod
    def random(n: int) -> TSPIntance:
        x = [uniform(0, 10) for _ in range(n)]
        y = [uniform(0, 10) for _ in range(n)]
        return TSPIntance(x=x, y=y)

In [11]:
class TSPSolution:
    tour: Tour
    cost: float

    def __init__(self, tour: Tour, **kwargs):
        assert 'cost' in kwargs or 'instance' in kwargs, \
            "You must pass the tour cost or a TSP instance to compute it"

        if 'cost' in kwargs:
            self.cost = kwargs.get('cost')
        elif 'instance' in kwargs:
            tsp = kwargs.get('instance')
            self.cost = sum(
                tsp.cost[i, j]
                for i in tour[:-1]
                for j in tour[1:]
            )

        self.tour = tour

    def __str__(self) -> str:
        return "[" + ', '.join(map(str, self.tour)) + f"] - Cost: {self.cost:.2f}"

In [12]:
class BranchAndCutFractionalSolver:
    tsp: TSPIntance
    m: Model
    x: tupledict
    G: Graph
    capacity: EdgePropertyMap

    def __init__(self, tsp: TSPIntance):
        self.tsp = tsp
        self.m = Model()
        self.x = self.m.addVars(self.tsp.arcs(), obj=self.tsp.cost, vtype=GRB.BINARY, name='x')
        self.__build_model()
        self.__build_graph()

    def __build_model(self) -> None:
        self.m.addConstrs(self.x.sum(i, '*') == 1 for i in self.tsp.vertices())
        self.m.addConstrs(self.x.sum('*', i) == 1 for i in self.tsp.vertices())

    def __build_graph(self) -> None:
        self.G = complete_graph(N=self.tsp.n, self_loops=False, directed=True)
        self.capacity = self.G.new_edge_property(value_type='double')
        self.G.edge_properties['capacity'] = self.capacity

    def solve(self) -> TSPSolution:
        self.m.setParam(GRB.Param.LazyConstraints, 1)
        self.m.optimize(lambda _, where: self.__separate(where=where))

        if self.m.Status != GRB.OPTIMAL:
            raise RuntimeError("Could not solve TSP model to optimality")
        
        return TSPSolution(tour=self.__tour_starting_at(0), cost=self.m.ObjVal)
    
    def __separate(self, where: int) -> None:
        if where not in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
            return
        
        # In MIPNODE, we must ensure that we have the optimal solution to the
        # linear relaxation at that node. Otherwise we are not "authorised"
        # to call cbGetNodeRel. See:
        # https://www.gurobi.com/documentation/9.1/refman/py_model_cbgetnoderel.html
        if where == GRB.Callback.MIPNODE and self.m.cbGet(GRB.Callback.MIPNODE_STATUS) != GRB.OPTIMAL:
            return
        
        self.__update_capacity()

        source = self.G.vertex(0)
        already_added = {source}

        for i in self.tsp.vertices():
            if i in already_added:
                continue

            target = self.G.vertex(i)
            residual = boykov_kolmogorov_max_flow(
                g=self.G, source=source, target=target,
                capacity=self.capacity
            )
            
            flow = residual.copy()
            flow.a = self.capacity.a - residual.a

            total_flow = sum(flow[a] for a in target.in_edges())

            if total_flow < 1 - 0.0001: # Avoid numerical issues
                print(f"Found a subset which violate a SEC (flow = {total_flow:.3f} < 1)")
                
                cut = min_st_cut(g=self.G, source=source, capacity=self.capacity, residual=residual)

                assert cut[source] == True
                assert cut[target] == False

                subtour = [j for j in self.tsp.vertices() if cut[self.G.vertex(j)] == False]

                assert len(subtour) < self.tsp.n

                self.__add_sec_for(subtour)

                already_added = already_added.union(subtour)

    def __update_capacity(self) -> None:
        for e in self.G.edges():
            i, j = e.source(), e.target()

            try: # If we are in MIPSOL
                xval = self.m.cbGetSolution(self.x[i,j])
            except: # If we are in MIPNODE
                xval = self.m.cbGetNodeRel(self.x[i,j])

            self.capacity[e] = xval

    def __tour_starting_at(self, i: Vertex) -> Tour:
        tour = [i]
        current = self.__next_vertex(i=i)

        while current != i:
            tour.append(current)
            current = self.__next_vertex(current)

        return tour

    def __next_vertex(self, i: Vertex) -> Vertex:
        for j in self.tsp.vertices():
            if j == i:
                continue

            try:
                # When in a callback
                x = self.m.cbGetSolution(self.x[i,j])
            except GurobiError:
                # When optimisation is over
                x = self.x[i,j].X

            if x > 0.5:
                return j
            
        raise RuntimeError(f"Vertex {i} has no successor!")
    
    def __add_sec_for(self, subtour: Tour) -> None:
        print("Adding subtour for [" + ', '.join(map(str, subtour)) + "]")
        self.m.cbLazy(
            sum(
                self.x[i, j]
                for i, j in self.tsp.arcs()
                if i in subtour and j not in subtour
            ) >= 1
        )

In [13]:
tsp = TSPIntance.random(n=20)
solver = BranchAndCutFractionalSolver(tsp=tsp)
solution = solver.solve()

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i7-6700K CPU @ 4.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 40 rows, 380 columns and 760 nonzeros
Model fingerprint: 0xd80a1258
Variable types: 0 continuous, 380 integer (380 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-01, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found a subset which violate a SEC (flow = 0.000 < 1)
Adding subtour for [4, 5, 6, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
Presolve time: 0.02s
Presolved: 40 rows, 380 columns, 760 nonzeros
Variable types: 0 continuous, 380 integer (380 binary)
Found a subset which violate a SEC (flow = 0.000 < 1)
Adding subtour for [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19]
Found a subset which violate a SEC (flow = 0.000 < 1)